<a href="https://colab.research.google.com/github/totheBrenda/portfolio/blob/main/sample.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**데이터 로딩 / 전처리**

In [ ]:
## 데이터로드
import pandas as pd
common_data = pd.read_csv('/content/common_data.csv').drop_duplicates(keep='first').reset_index(drop='true')
vktod = pd.read_csv('/content/vktod.csv')                          ## 걸러진거 (5843, 63)
dma_y_data = pd.read_csv('/content/dma_y_data.csv')
# p_data = pd.read_csv('/content/p_data.csv').drop_duplicates(keep='first').reset_index(drop='true')

In [ ]:
## 데이터 수 맞추고 합치고 뺄 거 빼고 전처리
df1 = pd.merge(vktod, common_data, left_on='%KEY', right_on='%KEY', how='left')
df2 = df1.drop(['순번', '지표명', 'TEST_RQST_YR', 'TEST_RQST_NO', 'TEST_VER_NO', 'DPRM_KOR_NM_ORG', 'CMPT_ID', 'MIX_PLC'], axis = 1)
df2['TD_NET_OIL'] = df2['TD_NET_OIL'].replace('-',0).astype(float)
# df2.info()

In [ ]:
## DMA Y 정리
dma_cols = dma_y_data.columns
dma_data = {}
dma_y_name, dma_data_shape = [], []
for dma_col in dma_cols[1:] :
  exec('dma_data[dma_col] = pd.merge(df2, dma_y_data[[\'%KEY\', dma_col]], left_on=\'%KEY\', right_on=\'%KEY\', how=\'left\').dropna()'.format(dma_col))
  dma_y_name.append(dma_col), dma_data_shape.append(dma_data[dma_col].shape)
pd.DataFrame({'dma_y_name' : dma_y_name, 'dma_data_shape' : dma_data_shape})

**타겟 선정 / 데이터 거르기**

In [ ]:
## 타겟 고르깅
target = 'target'
x_data = dma_data[target].drop(['%KEY', target], axis = 1).reset_index(drop=True)
y_data = dma_data[target][target].reset_index(drop=True)
x_data.shape, y_data.shape

**하나씩 걸러가기**

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

def arr_r2(x_data, y_data, fixed) :
  colname = []
  r2score = []
  coefvalue = []
  n = len(fixed)

  cols = x_data.drop(fixed, axis=1).columns

  for col in cols :

    fixed.append(col)
    model = LinearRegression()
    model.fit(x_data[fixed], y_data)

    y_pred = model.predict(x_data[fixed])
    coef = model.coef_[0]
    r2 = r2_score(y_data, y_pred)

    colname.append(col), r2score.append(r2), coefvalue.append(coef)
    fixed.pop()

  arr_df = pd.DataFrame({f'{target}_fix{n}_column' : colname, f'{target}_fix{n}_r2' : r2score})
  arr_df = arr_df.sort_values(f'{target}_fix{n}_r2', ascending=False).reset_index(drop=True)
  # tan0_5.shape
  return arr_df[:10]

In [ ]:
fixed = []
print(f'fixed column name : {fixed}')
f0 = arr_r2(x_data, y_data, fixed)
f0

In [ ]:
fixed = []
print(f'fixed column name : {fixed}')
f1 = arr_r2(x_data, y_data, fixed)
f1 = pd.concat([f0, f1], axis=1, join='inner')
f1

In [ ]:
fixed = []
print(f'fixed column name : {fixed}')
f2 = arr_r2(x_data, y_data, fixed)
f2 = pd.concat([f1, f2], axis=1, join='inner')
f2

In [ ]:
fixed = []
print(f'fixed column name : {fixed}')
f3 = arr_r2(x_data, y_data, fixed)
f3 = pd.concat([f2, f3], axis=1, join='inner')
f3

In [ ]:
fixed = []
print(f'fixed column name : {fixed}')
f4 = arr_r2(x_data, y_data, fixed)
f4 = pd.concat([f3, f4], axis=1, join='inner')
f4

In [ ]:
fixed = []
print(f'fixed column name : {fixed}')
f5 = arr_r2(x_data, y_data, fixed)
f5 = pd.concat([f4, f5], axis=1, join='inner')
f5

In [ ]:
fixed = []
print(f'fixed column name : {fixed}')
f6 = arr_r2(x_data, y_data, fixed)
f6 = pd.concat([f5, f6], axis=1, join='inner')
f6

In [ ]:
df.to_excel('title_arr_r2.xlsx')

**n개 랜덤으로 돌려서 r2 확인하기**

In [ ]:
## 랜덤만들기
import numpy as np

# np.random.seed(2)

def rd_arr_r2(x_data, y_data, rd_fixed, k, l) :
  colname = []
  r2score = []
  coefvalue = []
  n = len(rd_fixed)

  cols = x_data.drop(rd_fixed, axis=1).columns
  
  for i in range(k) :                           ## k : 몇번랜덤할끄야?
    rd_cols = np.random.choice(cols, l)         ## l : 몇개랜덤할끄야?

    for re_col in rd_cols :
      rd_fixed.append(re_col)

    model = LinearRegression()
    model.fit(x_data[rd_fixed], y_data)

    y_pred = model.predict(x_data[rd_fixed])
    coef = model.coef_[0]
    r2 = r2_score(y_data, y_pred)

    colname.append(rd_cols), r2score.append(r2), coefvalue.append(coef)
    
    for j in range(l) :
      rd_fixed.pop()
    

  rd_arr_df = pd.DataFrame({f'tan0_rd{n}_column' : colname, f'tan0_rd{n}_r2' : r2score})
  rd_arr_df = rd_arr_df.sort_values(f'tan0_rd{n}_r2', ascending=False).reset_index(drop=True)
  # tan0_5.shape
  return rd_arr_df[:10]

In [ ]:
rd_fixed =  []
rd_arr_r2(x_data, y_data, rd_fixed, 5, 2)           ## 4번째 : 몇번랜덤돌릴?  5번째 : 한번에몇개뽑을?

**plt 그리기**

In [ ]:
import matplotlib.pyplot as plt

def plts(x_data, y_data, pred_fixed, s, f) :
  model = LinearRegression()
  model.fit(x_data[pred_fixed], y_data)
  y_pred = model.predict(x_data[pred_fixed])
  
  fig, ax = plt.subplots(figsize=(20, 7))
  ax.set_title(f'green = y_data({target}),                    red = pred_data({pred_fixed})')
  ax.plot(y_data[s:f], label = 'val', marker = 'o', markersize = 5, color = 'green')
  ax.plot(y_pred[s:f], label = 'pred', marker='o', markersize = 5, color = 'red')

In [ ]:
pred_fixed = []
plts(x_data, y_data, pred_fixed, None, None);

**계수확인하기**

In [ ]:
## coef_
def values(x_data, y_data, pred_fixed) :
  model = LinearRegression()
  model.fit(x_data[pred_fixed], y_data)
  y_pred = model.predict(x_data[pred_fixed])

  coe = model.coef_
  df = pd.DataFrame({'fixed_column' : pred_fixed, 'coef' : coe})
  return df

In [ ]:
pred_fixed = []
coef_df = values(x_data, y_data, pred_fixed)
coef_df

In [ ]:
coef_df.to_excel('title_coef.xlsx', index=False)